# Grounded Theory Analysis: Professional Identity & AI

## Research Question
**"How do professionals negotiate maintaining their identity and legitimacy in the face of a tool that threatens to replace them?"**

---

### Dataset
- **Source**: [Anthropic/AnthropicInterviewer](https://huggingface.co/datasets/Anthropic/AnthropicInterviewer)
- **Split**: workforce (1,000 interviews)
- **Method**: Gioia Methodology for Grounded Theory
- **Model**: Gemini 2.5 Flash

---

## 1. Installation des dépendances

In [ ]:
!pip install -q google-generativeai
!pip install -q datasets
!pip install -q pandas
!pip install -q tqdm

## 2. Configuration

In [ ]:
import google.generativeai as genai
from datasets import load_dataset
import json
import re
from typing import List, Dict, Any
import pandas as pd
from tqdm.notebook import tqdm
import time
import random

# ============================================
# CONFIGURATION
# ============================================

# Clé API Gemini - UTILISER LES SECRETS COLAB
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Modèle Gemini
MODEL_NAME = "gemini-2.5-flash"

# Question de recherche
RESEARCH_QUESTION = """How do professionals negotiate maintaining their identity and legitimacy 
in the face of a tool that threatens to replace them?"""

# Nombre d'interviews à analyser
SAMPLE_SIZE = 50  # Mettre None pour tout analyser (1000)

# Traitement: 1 interview à la fois (plus fiable pour interviews longues)
BATCH_SIZE = 1

# Limite de caractères par interview (pour éviter erreur 400)
MAX_CHARS_PER_INTERVIEW = 8000

# Configuration API
genai.configure(api_key=GOOGLE_API_KEY)

print(f"✅ Configuration chargée")
print(f"   Modèle: {MODEL_NAME}")
print(f"   Sample: {SAMPLE_SIZE or 'Tous (1000)'}")
print(f"   Max chars/interview: {MAX_CHARS_PER_INTERVIEW}")

## 3. Chargement du Dataset

In [ ]:
# Charger le dataset depuis Hugging Face
print("📥 Chargement du dataset Anthropic/AnthropicInterviewer...")
dataset = load_dataset("Anthropic/AnthropicInterviewer", split="workforce")

print(f"\n✅ Dataset chargé: {len(dataset)} interviews")
print(f"\n📋 Colonnes disponibles: {dataset.column_names}")

# Aperçu
print("\n📄 Exemple d'interview:")
print(dataset[0])

In [ ]:
# Préparer les données pour l'analyse
def prepare_interviews(dataset, sample_size=None):
    """Prépare les interviews pour l'analyse (simplifié pour interviews courtes)."""
    interviews = []
    
    # Échantillonner si nécessaire
    indices = list(range(len(dataset)))
    if sample_size and sample_size < len(dataset):
        indices = random.sample(indices, sample_size)
    
    for idx in indices:
        item = dataset[idx]
        
        interview = {
            'id': idx,
            'content': '',
        }
        
        # Extraire le contenu textuel selon la structure du dataset
        if isinstance(item, dict):
            # Chercher les champs de texte courants
            for key in ['text', 'content', 'transcript', 'conversation', 'messages']:
                if key in item and item[key]:
                    val = item[key]
                    if isinstance(val, str):
                        interview['content'] = val
                    elif isinstance(val, list):
                        # Liste de messages -> joindre
                        interview['content'] = "\n".join([
                            f"{m.get('role', 'unknown')}: {m.get('content', str(m))}" 
                            if isinstance(m, dict) else str(m) 
                            for m in val
                        ])
                    break
            
            # Fallback: utiliser tout le dict
            if not interview['content']:
                interview['content'] = json.dumps(item, ensure_ascii=False)
        else:
            interview['content'] = str(item)
        
        interviews.append(interview)
    
    return interviews

# Préparer les interviews
interviews = prepare_interviews(dataset, SAMPLE_SIZE)

print(f"✅ {len(interviews)} interviews préparées")
print(f"\n📏 Longueur moyenne: {sum(len(i['content']) for i in interviews) // len(interviews)} caractères")
print(f"\n📄 Aperçu interview #1 (500 chars):")
print("-"*40)
print(interviews[0]['content'][:500] + "...")

## 4. Service Gemini

In [ ]:
class GeminiService:
    """Service pour les appels à l'API Gemini avec gestion des erreurs."""
    
    def __init__(self, model_name: str = MODEL_NAME):
        self.model = genai.GenerativeModel(model_name)
        self.request_count = 0
        self.last_request_time = 0
    
    def _rate_limit(self):
        """Gestion du rate limiting."""
        self.request_count += 1
        # Pause toutes les 10 requêtes pour éviter les limites
        if self.request_count % 10 == 0:
            time.sleep(2)
    
    def generate(self, prompt: str, temperature: float = 0.7, max_retries: int = 3) -> str:
        """Génère une réponse avec retry automatique."""
        self._rate_limit()
        
        config = genai.types.GenerationConfig(
            temperature=temperature,
            top_p=0.95,
            max_output_tokens=8192,
        )
        
        for attempt in range(max_retries):
            try:
                response = self.model.generate_content(prompt, generation_config=config)
                return response.text
            except Exception as e:
                if attempt < max_retries - 1:
                    wait_time = (attempt + 1) * 5
                    print(f"⚠️ Erreur API, retry dans {wait_time}s: {str(e)[:100]}")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Échec après {max_retries} tentatives: {e}")
                    return ""
    
    def generate_json(self, prompt: str, temperature: float = 0.3) -> Any:
        """Génère une réponse JSON structurée."""
        json_prompt = f"""{prompt}

CRITICAL: Respond ONLY with valid JSON. No text before or after.
Do not use ```json or ``` markers.
"""
        response = self.generate(json_prompt, temperature=temperature)
        
        if not response:
            return None
        
        # Nettoyer la réponse
        cleaned = response.strip()
        if cleaned.startswith('```json'):
            cleaned = cleaned[7:]
        if cleaned.startswith('```'):
            cleaned = cleaned[3:]
        if cleaned.endswith('```'):
            cleaned = cleaned[:-3]
        
        try:
            return json.loads(cleaned.strip())
        except json.JSONDecodeError as e:
            print(f"⚠️ JSON parsing error: {e}")
            # Essayer d'extraire le JSON
            match = re.search(r'[\[\{].*[\]\}]', cleaned, re.DOTALL)
            if match:
                try:
                    return json.loads(match.group())
                except:
                    pass
            return None

# Instance globale
gemini = GeminiService()
print("✅ GeminiService initialisé")

## 5. Codage Gioia - Adapté à la Question de Recherche

In [ ]:
class GioiaAnalysis:
    """Analyse Gioia optimisée - traite une interview à la fois pour plus de fiabilité."""
    
    def __init__(self, gemini_service: GeminiService, research_question: str):
        self.gemini = gemini_service
        self.research_question = research_question
    
    def clean_text(self, text: str) -> str:
        """Nettoie le texte pour éviter les erreurs API."""
        # Supprimer les caractères de contrôle problématiques
        text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', text)
        # Limiter la longueur
        return text[:MAX_CHARS_PER_INTERVIEW]
    
    def code_single_interview(self, interview: Dict) -> List[str]:
        """Code une seule interview."""
        content = self.clean_text(interview['content'])
        
        prompt = f"""You are an expert qualitative researcher using the Gioia methodology.

RESEARCH QUESTION: {self.research_question}

INTERVIEW TRANSCRIPT:
{content}

Analyze this interview and identify FIRST-ORDER CODES (5-12 codes).
Focus on:
- Professional identity and self-definition
- Perceived threats from AI
- Coping and adaptation strategies
- Legitimacy claims and value justification
- Identity negotiation with technology

First-order codes should be close to the informant's own words.

Respond ONLY with a JSON array of strings:
["code 1", "code 2", "code 3"]
"""
        result = self.gemini.generate_json(prompt)
        return result if isinstance(result, list) else []
    
    def initial_coding(self, interviews: List[Dict]) -> tuple:
        """Codage de premier ordre - une interview à la fois."""
        coded_interviews = []
        all_codes = []
        
        for interview in tqdm(interviews, desc="Coding interviews"):
            codes = self.code_single_interview(interview)
            interview['first_order_codes'] = codes
            coded_interviews.append(interview)
            all_codes.extend(codes)
            
            # Petite pause pour éviter rate limiting
            time.sleep(0.5)
        
        return coded_interviews, list(set(all_codes))
    
    def second_order_coding(self, first_order_codes: List[str]) -> Dict[str, List[str]]:
        """Codage de second ordre - thèmes émergents."""
        # Limiter le nombre de codes envoyés
        codes_to_send = first_order_codes[:150]
        
        prompt = f"""You are an expert qualitative researcher using the Gioia methodology.

RESEARCH QUESTION: {self.research_question}

FIRST-ORDER CODES ({len(codes_to_send)} codes from professional interviews about AI):
{json.dumps(codes_to_send, ensure_ascii=False, indent=2)}

Group these into 6-10 SECOND-ORDER THEMES.
Second-order themes are more abstract concepts capturing:
- Identity maintenance strategies
- Threat perception patterns
- Legitimacy construction mechanisms
- Adaptation behaviors
- Professional boundary work

Respond with a JSON object:
{{"Theme Name": ["code1", "code2"], "Another Theme": ["code3", "code4"]}}
"""
        result = self.gemini.generate_json(prompt)
        return result if isinstance(result, dict) else {}
    
    def aggregate_dimensions(self, second_order_codes: Dict[str, List[str]]) -> Dict[str, List[str]]:
        """Dimensions agrégées - concepts théoriques."""
        prompt = f"""You are an expert qualitative researcher using the Gioia methodology.

RESEARCH QUESTION: {self.research_question}

SECOND-ORDER THEMES:
{json.dumps(second_order_codes, ensure_ascii=False, indent=2)}

Aggregate these into 3-5 AGGREGATE DIMENSIONS.
Aggregate dimensions are high-level theoretical constructs that answer the research question.

Respond with a JSON object:
{{"Dimension Name": ["Theme A", "Theme B"], "Another Dimension": ["Theme C"]}}
"""
        result = self.gemini.generate_json(prompt)
        return result if isinstance(result, dict) else {}
    
    def run_full_analysis(self, interviews: List[Dict]) -> Dict:
        """Exécute l'analyse Gioia complète."""
        print(f"\n{'='*60}")
        print("🔬 GIOIA ANALYSIS - Professional Identity & AI")
        print(f"{'='*60}")
        print(f"\n📋 Research Question:\n{self.research_question}\n")
        print(f"📊 {len(interviews)} interviews to analyze")
        
        # Étape 1: Codage initial
        print("\n📝 STEP 1: First-Order Coding...")
        coded_interviews, unique_first_order = self.initial_coding(interviews)
        print(f"   ✅ {len(unique_first_order)} unique first-order codes identified")
        
        # Étape 2: Codage de second ordre
        print("\n📝 STEP 2: Second-Order Coding...")
        second_order = self.second_order_coding(unique_first_order)
        print(f"   ✅ {len(second_order)} second-order themes created")
        
        # Étape 3: Dimensions agrégées
        print("\n📝 STEP 3: Aggregate Dimensions...")
        dimensions = self.aggregate_dimensions(second_order)
        print(f"   ✅ {len(dimensions)} aggregate dimensions created")
        
        return {
            'interviews': coded_interviews,
            'first_order_codes': unique_first_order,
            'second_order_themes': second_order,
            'aggregate_dimensions': dimensions
        }

# Instance
gioia = GioiaAnalysis(gemini, RESEARCH_QUESTION)
print("✅ GioiaAnalysis initialisé")

## 6. Lancer l'Analyse

In [ ]:
# Exécuter l'analyse Gioia complète
results = gioia.run_full_analysis(interviews)

## 7. Visualisation des Résultats

In [ ]:
from IPython.display import display, HTML, Markdown

def display_gioia_structure(results: Dict):
    """Affiche la structure Gioia complète."""
    dimensions = results.get('aggregate_dimensions', {})
    second_order = results.get('second_order_themes', {})
    
    html = "<style>"
    html += ".gioia-table { border-collapse: collapse; width: 100%; font-family: Arial, sans-serif; }"
    html += ".gioia-table th, .gioia-table td { border: 1px solid #ddd; padding: 10px; text-align: left; vertical-align: top; }"
    html += ".gioia-table th { background-color: #4a90d9; color: white; }"
    html += ".dim-cell { background-color: #e8f4e8; font-weight: bold; font-size: 14px; }"
    html += ".theme-cell { background-color: #fff8e8; }"
    html += ".code-cell { font-size: 12px; color: #555; }"
    html += "</style>"
    
    html += "<h2>📊 Gioia Data Structure</h2>"
    html += "<table class='gioia-table'>"
    html += "<tr><th>1st Order Codes</th><th>2nd Order Themes</th><th>Aggregate Dimensions</th></tr>"
    
    for dim_name, themes in dimensions.items():
        first_row_dim = True
        theme_count = len(themes)
        
        for theme in themes:
            first_codes = second_order.get(theme, [])
            codes_html = "<br>".join([f"• {c}" for c in first_codes[:6]])
            if len(first_codes) > 6:
                codes_html += f"<br><i>... +{len(first_codes)-6} more</i>"
            
            html += "<tr>"
            html += f"<td class='code-cell'>{codes_html}</td>"
            html += f"<td class='theme-cell'>{theme}</td>"
            
            if first_row_dim:
                html += f"<td class='dim-cell' rowspan='{theme_count}'>{dim_name}</td>"
                first_row_dim = False
            
            html += "</tr>"
    
    html += "</table>"
    display(HTML(html))

# Afficher la structure
display_gioia_structure(results)

In [ ]:
# Afficher les statistiques
print("\n" + "="*60)
print("📊 ANALYSIS SUMMARY")
print("="*60)
print(f"\n📋 Interviews analyzed: {len(results['interviews'])}")
print(f"📝 First-order codes: {len(results['first_order_codes'])}")
print(f"🏷️ Second-order themes: {len(results['second_order_themes'])}")
print(f"📦 Aggregate dimensions: {len(results['aggregate_dimensions'])}")

print("\n\n📦 AGGREGATE DIMENSIONS:")
print("-"*40)
for dim, themes in results['aggregate_dimensions'].items():
    print(f"\n🔹 {dim}")
    for theme in themes:
        code_count = len(results['second_order_themes'].get(theme, []))
        print(f"   └─ {theme} ({code_count} codes)")

## 8. Construction du Modèle Théorique

In [ ]:
def build_theoretical_model(results: Dict, research_question: str) -> Dict:
    """Construit un modèle théorique à partir des résultats Gioia."""
    
    prompt = f"""You are a qualitative research expert building grounded theory.

RESEARCH QUESTION:
{research_question}

GIOIA ANALYSIS RESULTS:

Aggregate Dimensions:
{json.dumps(results['aggregate_dimensions'], ensure_ascii=False, indent=2)}

Second-Order Themes:
{json.dumps(results['second_order_themes'], ensure_ascii=False, indent=2)}

Based on this analysis, construct a THEORETICAL MODEL that:

1. Answers the research question
2. Identifies key constructs and their relationships
3. Proposes testable propositions
4. Connects to existing literature on:
   - Professional identity theory
   - Legitimacy theory
   - Technology acceptance/resistance
   - Boundary work in professions

Respond with a JSON object:
{{
  "model_name": "Name of your theoretical model",
  "core_argument": "The central theoretical argument (2-3 sentences)",
  "constructs": [
    {{
      "name": "Construct name",
      "definition": "Definition",
      "type": "independent/dependent/mediator/moderator/process",
      "grounded_in": ["Aggregate dimension(s) it comes from"]
    }}
  ],
  "propositions": [
    "P1: First proposition...",
    "P2: Second proposition..."
  ],
  "theoretical_contributions": [
    "Contribution 1",
    "Contribution 2"
  ],
  "practical_implications": [
    "Implication 1",
    "Implication 2"
  ],
  "boundary_conditions": [
    "When/where this model applies"
  ],
  "future_research": [
    "Direction 1",
    "Direction 2"
  ]
}}
"""
    
    model = gemini.generate_json(prompt)
    return model if isinstance(model, dict) else {}

# Construire le modèle
print("🏗️ Building theoretical model...")
theoretical_model = build_theoretical_model(results, RESEARCH_QUESTION)
print("✅ Model constructed")

In [ ]:
# Afficher le modèle théorique
def display_theoretical_model(model: Dict):
    md = f"""# 🧠 {model.get('model_name', 'Theoretical Model')}

## Core Argument
{model.get('core_argument', 'N/A')}

---

## Theoretical Constructs
"""
    for c in model.get('constructs', []):
        md += f"\n### {c.get('name')} *({c.get('type')})*\n"
        md += f"{c.get('definition')}\n"
        md += f"- *Grounded in: {', '.join(c.get('grounded_in', []))}*\n"
    
    md += "\n---\n\n## Propositions\n"
    for p in model.get('propositions', []):
        md += f"- **{p}**\n"
    
    md += "\n---\n\n## Theoretical Contributions\n"
    for c in model.get('theoretical_contributions', []):
        md += f"- {c}\n"
    
    md += "\n## Practical Implications\n"
    for i in model.get('practical_implications', []):
        md += f"- {i}\n"
    
    md += "\n## Boundary Conditions\n"
    for b in model.get('boundary_conditions', []):
        md += f"- {b}\n"
    
    md += "\n## Future Research Directions\n"
    for f in model.get('future_research', []):
        md += f"- {f}\n"
    
    display(Markdown(md))

display_theoretical_model(theoretical_model)

## 9. Visualisation du Modèle (Mermaid)

In [ ]:
def generate_model_diagram(model: Dict) -> str:
    """Génère un diagramme Mermaid du modèle."""
    prompt = f"""Create a Mermaid flowchart diagram for this theoretical model:

{json.dumps(model, ensure_ascii=False, indent=2)}

Requirements:
- Use graph LR (left to right) or graph TD (top down)
- Show all constructs as nodes
- Show relationships based on propositions as labeled arrows
- Use different node shapes for different construct types:
  - Independent variables: rectangles
  - Dependent variables: rounded rectangles
  - Mediators: circles
  - Moderators: diamonds
- Add colors using style definitions

Return ONLY the Mermaid code, no markdown markers.
"""
    
    response = gemini.generate(prompt, temperature=0.3)
    
    # Nettoyer
    cleaned = response.strip()
    if cleaned.startswith('```mermaid'):
        cleaned = cleaned[10:]
    if cleaned.startswith('```'):
        cleaned = cleaned[3:]
    if cleaned.endswith('```'):
        cleaned = cleaned[:-3]
    
    return cleaned.strip()

# Générer le diagramme
print("📊 Generating model diagram...")
mermaid_code = generate_model_diagram(theoretical_model)
print("\n📈 Mermaid Diagram Code:")
print("-"*40)
print(mermaid_code)

In [ ]:
# Afficher le diagramme (fonctionne dans Colab/Jupyter)
from IPython.display import display, HTML

html = f"""
<script src="https://cdn.jsdelivr.net/npm/mermaid/dist/mermaid.min.js"></script>
<script>mermaid.initialize({{startOnLoad:true, theme:'default'}});</script>
<div class="mermaid">
{mermaid_code}
</div>
"""
display(HTML(html))

## 10. Export des Résultats

In [ ]:
from datetime import datetime

# Compiler tous les résultats
final_results = {
    'metadata': {
        'research_question': RESEARCH_QUESTION,
        'dataset': 'Anthropic/AnthropicInterviewer',
        'split': 'workforce',
        'sample_size': len(interviews),
        'model': MODEL_NAME,
        'analysis_date': datetime.now().isoformat()
    },
    'gioia_analysis': {
        'first_order_codes': results['first_order_codes'],
        'second_order_themes': results['second_order_themes'],
        'aggregate_dimensions': results['aggregate_dimensions']
    },
    'theoretical_model': theoretical_model,
    'visualization': {
        'mermaid_diagram': mermaid_code
    },
    'coded_interviews': [
        {
            'id': i['id'],
            'first_order_codes': i.get('first_order_codes', [])
        } for i in results['interviews']
    ]
}

# Sauvegarder
filename = f"grounded_theory_professional_identity_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(final_results, f, ensure_ascii=False, indent=2)

print(f"\n✅ Results saved to: {filename}")

# Télécharger (Colab)
try:
    from google.colab import files
    files.download(filename)
    print("📥 Download started...")
except:
    print("(Manual download required outside Colab)")

In [ ]:
# Export CSV des codes pour analyse supplémentaire
codes_data = []
for theme, codes in results['second_order_themes'].items():
    # Trouver la dimension agrégée
    dimension = None
    for dim, themes in results['aggregate_dimensions'].items():
        if theme in themes:
            dimension = dim
            break
    
    for code in codes:
        codes_data.append({
            'first_order_code': code,
            'second_order_theme': theme,
            'aggregate_dimension': dimension
        })

df_codes = pd.DataFrame(codes_data)
csv_filename = f"gioia_codes_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_codes.to_csv(csv_filename, index=False)
print(f"\n✅ Codes exported to: {csv_filename}")

# Afficher un aperçu
display(df_codes.head(20))

---
## 📖 Notes

### Méthode Gioia
La méthode Gioia est une approche rigoureuse pour développer une théorie ancrée:
1. **Codes de 1er ordre**: Proches des mots des participants
2. **Thèmes de 2nd ordre**: Concepts plus abstraits du chercheur
3. **Dimensions agrégées**: Construits théoriques de haut niveau

### Personnalisation
- Modifiez `SAMPLE_SIZE` pour analyser plus/moins d'interviews
- Adaptez `RESEARCH_QUESTION` pour changer le focus
- Le modèle peut être changé dans la configuration

### Références
- Gioia, D. A., Corley, K. G., & Hamilton, A. L. (2013). Seeking qualitative rigor in inductive research
- Dataset: [Anthropic/AnthropicInterviewer](https://huggingface.co/datasets/Anthropic/AnthropicInterviewer)